# Наследование

## Наследование

Наследование — это механизм, позволяющий создавать производные классы, расширяя уже существующие.

    struct Person {
        string name()  const {  // NB: const метод доступен, когда экземпляр константа 
            return name_;       // + const метод не может менять поля, т.к. this -> const
        }
        int age() const { 
            return age_; 
        }
    private:
        string  name_;
        int  age_;
    };
        
    struct  Student : Person {
        string  university () const { 
            return  uni_; 
        }
    private:
        string  uni_;
    };

## Класс-наследник 

У объектов класса-наследника можно вызывать публичные методы родительского класса.
    
    Student s;
    cout  << s.name() << endl
          << s.age()  << endl
          << s.university () << endl;
    
Внутри объекта класса-наследника хранится экземпляр родительского класса.

    [(name_, age_) Person]   [ ( [name_, age_) Person ], _uni_) Student]

## Создание/удаление объекта производного класса 

При создании объекта производного класса сначала вызывается конструктор родительского класса. Иногда его нужно вызывать самостоятельно (например, если у родительского нет конструктора по умолчанию, а есть только с параметрами)

    struct  Person {
        Person(string  name , int age)
        : name_(name), age_(age)
        {}
        ...
        };
    
    struct  Student : Person {
        Student(string  name , int age , string  uni)
        : Person(name , age), uni_(uni)                     // как бы "вызов" родительского конструктора (*)
        {}
        ...
    };

*) список инициализации это не вызов по сути, а инструкции компилятору, что он должен делать при создании объекта. Когда мы инициализируем члены класса в списке инициализации, мы ожидаем, что они уже будут создаваться с этими значениями, а не то, что они будут сначала созданы, а потом им эти значения присвоены (в этом основное отличие от присваивания значений в теле конструктора).

Если есть (публичный?) конструктор по умолчанию (который без параметров), то он вызовется компилятором.  Как следствие, передача параметров в родительский конструктор может быть проведена только в списке инициализации. Это особенность синтаксиса. Когда начинается выполнение тела конструктора, конструктор базового класса уже вызван. Поэтому передавать параметры нужно до входа в тело конструктора.   


После деструктора Student вызывается деструктор Person. Более того, повлиять на это нельзя, эти вызовы кодирует компилятор.

## Приведения 

Для производных классов определены следующие приведения:

    Student s("Alex", 21, "Oxford");
    Person & l = s;   // автоматически будет приведен тип Student & -> Person &
    Person * r = &s;  // ... Student * -> Person *

*) Это автоматическое приведение типа при создании объектов разрешено только из потомков в родителей, а не наоборот. Очевидно почему.  
    
Поэтому объекты класса-наследника могут присваиваться объектам родительского класса (а также использоваться для инициализации):

    Student s("Alex", 21, "Oxford");
    Person p = s; // Person("Alex", 21);

При этом копируются только поля класса-родителя (срезка). (Т.е. в данном случае вызывается конструктор копирования Person(Person const& p), которому в качестве ссылки &p подсовывается Student* (тип ссылки приводится автоматически), и который не знает про uni_.)

## Модификатор доступа protected

∙Класс-наследник не имеет доступа к private-членам родительского класса.  
∙Для определения закрытых членов класса доступных только наследникам используется модификатор protected.

    struct  Person {
        ...
    protected:
        string  name_;
        int  age_;
    };
    
    struct  Student : Person {
        ... // можно менять поля name_ и age_
    };

\*) применение модификатора protected к полям (не методам) потенциально нарушает инкапсуляцию, т.е. в этом случае наследники имеют возможность нарушить инварианты класса. Не стоит так делать.

Смысл protected - выделение специального интерфейса для наследников, когда у класса есть публичный интерфейс и защищенный интерфейст для работы с производными классами.

В немного более общем случае синтаксис наследования выглядит следующим образом:

    struct Derived : <modifier> Base { };

где <modifier> — это одно из ключевых слов public, protected или private. Нас на данном этапе интересует только публичное (public) наследование, поэтому смысл других модификаторов не столь важен. Если модификатор не указан явно, то используется модификатор по-умолчанию, и важно знать, что для структур по-умолчанию используется модификатор public (как раз то, что нам нужно), т.е. в примере:

    struct Student : Person { };

Student публично (public) наследуется от класса Person. А для классов (в противовес структурам) по-умолчанию используется модификатор private, т.е. в примере:

    class Student : Person { };

Student приватно (private) наследуется от класса Person (не то, чего мы хотим). Чтобы унаследовать класс Student от класса Person публично (то, что мы хотим), необходимо явно указать модификатор:

    class Student : public Person { };

При этом не важно, является ли Person классом или структурой, важно лишь то, является ли наследник (Student) структурой или классом (в терминах C++). 

# Перегрузка

## Перегрузка функций

В отличие от C в C++ можно определить несколько функций с одинаковым именем, но разными параметрами.

    double  square(double d) { return d * d; }  
    int     square(int    i) { return i * i; }

При вызове функции по имени будет произведен поиск наиболее подходящей функции (candidate):

    int      a = square (4);       // square(int)
    double   b = square (3.14);    // square(double)
    double   c = square (5);       // square(int) - тип возвращаемого значения при выборе candidate не учитывается
    int      d = square(b);        // square(double)
    float    e = square (2.71f);   // square(double) - компилятор преобразует float to double

## Перегрузка методов

    struct  Vector2D {
        Vector2D(double x, double y) : x(x), y(y) {}
        
        Vector2D  mult(double d) const
            { return  Vector2D(x * d, y * d); }
        
        double   mult(Vector2D  const& p) const     // аргумент по конст.ссылке (*)
            { return x * p.x + y * p.y;    }
        
        double x, y;
    };
    
    Vector2D p(1, 2);
    Vector2D q = p.mult(10); // (10, 20)
    double   r = p.mult(q);   // 50

\*) еще раз для закрепления конст.ссылка -> аргумент предохраняется от изменения + т.к. при передаче аргумента в функцию всегда происходит копирование, то если аргумент при этом ссылка (указатель), то копируется просто адрес, и с завершением времени жизни объекта, выступающего в роли аргумента, в функции по его адресу был бы мусор, если бы конст.ссылка не заставила компилятор предотвратить (неявный) вызов деструктора объекта, выступающего в роли аргумента.

## Перегрузка при наследовании

    struct  File {
        void  write(char  const * s);
        ...
    };
    
    struct  FormattedFile : File {
        void  write(int i);
        void  write(double d);
        using  File::write; // означает разрешение на использование методов базового класса
        ...
    };
    
    FormattedFile f;
    f.write(4);             
    f.write("Hello");       // возьмет из базового класса (т.к. есть using File::write;)

## Правила перегрузки

1. Если есть точное совпадение (среди функций, объявленных до (!) такого поиска), то используется оно.  
2. Если нет функции, которая могла бы подойти с учётом преобразований, выдаётся ошибка.  
3. Есть функции, подходящие с учётом преобразований:
    
    3.1 Расширение типов (встроенные преобразования от меньшего типа к большему).
        
        char, signed char, short → int
        unsigned char, unsigned short → int/unsigned int
        float → double
        
    3.2 Стандартные преобразования (числа (напр. double -> int), указатели (напр. (int * ) -> (void *) или указатель на производный к указателю на базовый класс)).  
    3.3 Пользовательские преобразования (не работают, если объявлен explicit - запрет неявных вызовов). Напр. если у класса А определить конструктор от типа Б, то будет произведено преобразование Б -> А.
    
    В случае нескольких параметров нужно, чтобы выбранная функция была строго лучше остальных, т.е. по каждому параметру лучший кандидат должен быть не хуже других в смысле п.3 и по какому-то параметру - строго лучше. Если такой кандидат один, то он будет выбран, иначе - ошибка компиляции "неоднозначный вызов".

NB: перегрузка выполняется на этапе компиляции. При работе программы уже никаких перегрузок нет, вызываются конкретные функции по конкретным адресам в памяти.
NB: поэтому с перегрузкой лучше не перебарщивать.

# Виртуальные методы

## Переопределение методов (overriding)

Отличие от перегрузки в том, что сигнатура не меняется, как при перегрузке.

    struct  Person {
        string  name()  const { return  name_; }
        ...
    };
    
    struct  Professor : Person {
        string  name()  const {
            return "Prof. " + Person::name ();      // + метод из базового класса
        }
        ...
    };
    
    Professor  pr("Stroustrup");
    cout  << pr.name() << endl; // Prof. Stroustrup
    Person * p = &pr;           // ук.типа Person, иниц.(относится только к полям) объектом типа Professor (со срезкой или без, неважно)
    cout  << p->name() << endl; // Stroustrup (рассматриваются только методы Person)

\*) в памяти на стеке вызовов будет объект pr типа Professor, объект указателя p. Объект типа Person, доступный по указателю p, будет в дин.памяти (cтруктура, отвечающая за выделение дополнительной памяти, называется кучей). Как то так вроде...

\**) p указывает на место, где лежат данные объекта pr, но функции (т.е. методы там не лежат). Компилятор выискивая метод для p->name() наткнется на таковой в определении типа Person и захардкодит его в ассемблер. Как то так вроде...


## Виртуальные методы

    struct  Person {
        virtual  string  name() const { return  name_; }    // virtual означает, что в расчет берется тип объекта, на который ссылается указатель this
        ...
    };
    
    struct  Professor : Person {
        string  name()  const {
            return "Prof. " + Person ::name ();
        }
    ...
    };
    
    Professor  pr("Stroustrup");
    cout  << pr.name() << endl; // Prof. Stroustrup
    Person * p = &pr;             // на тип Professor
    cout  << p->name() << endl; // Prof. Stroustrup     (this найдет у себя такой метод)

\*) virtual определяет, также, что конкретный метод (адрес конкретной функции в памяти) не хардкодится компилятором, а будет определен на этапе выполнения (с помощью таблицы виртуальных методов), в зависимости от типа объекта на который ссылается указатель, который запросил метод.

### Виртуальные методы, это прежде всего методы, поэтому:

    Все виртуальные методы базового класса являются виртуальными и для производных классов.
    Обращение к объекту по ссылке также позволяет вызывать виртуальные методы.
    Виртуальные методы могут быть константными.
    Невиртуальные методы могут вызывать виртуальные методы.
    Виртуальный метод можно вызвать не только через указатель на объект.
    Для того, чтобы вызвать виртуальный метод, объект не обязательно должен быть создан через new.
    Виртуальные методы могут вызывать невиртуальные методы. 

## Чистые виртуальные (абстрактные) методы

Особый вид виртуальных методов. Как их использовать?  

    struct  Person {
        virtual  string  occupation () const = 0;   // =0 специальное обозначение отсуствия реализации
        // хотя бы 1 метод с таким определением (=0) и класс становится абстрактным
        ...
    };

    struct  Student : Person {
        string  occupation () const {
            return "student";
        }
        ...
    };

    struct  Professor : Person {
        string  occupation () const {return "professor";}
        ...
    };
    
    Person * p = next_person ();  // пусть есть некторая функция (например итераторного типа) возвращает указатель
                                  // базового типа на экземпляры различных унаследованных типов, где конечно же
                                  // виртуальные методы невиртуально определены
    cout  << p->occupation ();    // тогда получаем возможность единым образом, по однотипному указателю, без 
                                  // приведения типов обрабатывать объекты разных классов

Компилятор физически не сможет и не станет создавать экземпляры классов, в которых определены абстрактные методы (из-за неопределенности, чего писать в код его методов). Т.е. идейно, это такой запрет создавать экземпляры класса в принципе. Но не запрет унаследоваться от него и определять методы там, а также не запрещено создавать указатели такого типа.

NB: стиль написания. Считается хорошим тоном в унаследованных классах переопределенные виртуальные методы также обозначать как virtual (просто дать понять, что этот метод переопределяет родительский без необходимости лезть в код родительского класса). Метод виртуальные по факту, поэтому синтаксически это избыточно, но некоторые так делают.  
Кроме того, есть ключевое слово GetWeapon() override; которое помогает программисту не ошибиться с сигнатурой переопределяемого метода.  С этим словом компилятор выдаст ошибку "marked override, but does not override". В то время как без override он просто молча создаст отдельный метод с новой, отличной от переопределяемого метода, сигнатурой.

\*) Кстати, начиная с C++11 можно явно запретить наследоваться от классов, используя ключевое слово final. Например, стандартная библиотека не подразумевает наследования

## Виртуальный деструктор

К чему приведёт такой код?

    struct  Person {
        ...
    };

    struct  Student : Person {
        ...
    private:
        string  uni_;           // расширили базовый класс
    };
    
    int  main() {
        Person * p = new  Student("Alex" ,21,"Oxford"); // новый экз.унаследованного класса, 
                                                        // сохраняем через указатель на базовый класс
        ...
        delete p;       // деструктор по умолчанию, который создает компилятор ~Person (не виртуальный)
                        // ничего не знает про поле uni_, поэтому оно попадет в утечку памяти
    }

Правильная реализация - если есть виртуальные методы в базовом классе, то и обозначить его деструктор как виртуальный (даже пустой):

    struct  Person {
        ...
        virtual ~Person () {}
    };

    struct  Student : Person {
        ...
    private:
        string  uni_;
    };
    
    int  main() {
        Person * p = new  Student("Alex" ,21,"Oxford");
        ...
        delete p;       // по таблице виртуальных методов будет выбран деструктор ~Sudent, который как 
                        // минимум обнулит свои поля, потом вызовет родительский деструктор, и т.д.
                        // так что даже пустой виртуальный деструктор как миним поля освободит
                        // а так, мало ли чего там методы нааллоцируют в памяти, если дело касается работы с        
                        // динамической памятью, надо все аккуратно смотреть и освобождать
    }

В виде правила:

1. Если твой базовый класс предназначен для полиморфного использования, т.е. ты в нем определил виртуальные функции, то в этом случае деструктор обязан быть виртуальным.
2. Если базовый класс не предназначен для полиморфного использования, т.е. в нем отсутствуют какие-либо виртуальные функции, то тогда деструктор должен быть НЕ виртуальным.



## Полиморфизм

Полиморфизм - Возможность единообразно обрабатывать разные типы данных.  

Перегрузка функций - Выбор функции происходит в момент компиляции на основе типов аргументов функции (cтатический полиморфизм или полиморфизм времени компиляции).  

Виртуальные методы - Выбор метода происходит в момент выполнения на основе типа объекта, у которого вызывается виртуальный метод (динамический полиморфизм времени исполнения).


Также можно сказать, конкретными форматами реализации полиморфизма в С++ являются:

    void* - указатели типа void, которые можно приводить к нужному виду (unsafe)
    шаблоны (safe)
    ключевое слово overload - статический, времени компиляции
    ключевое слово override - динамический, времени выполнения


# Таблица виртуальных методов

## Таблица виртуальных методов

∙Динамический полиморфизм (полиморфизм вверемени выполнения) реализуется при помощи таблиц виртуальных методов.  
∙Таблица заводится для каждого полиморфного класса (таких у которых есть виртуальные методы). Создается для каждого класса, а не объекта.  
∙Объекты полиморфных классов содержат указатель на таблицу виртуальных методов соответствующего класса. Указатель добавляется на этапе компиляции. Это такое неявное поле vptr, к которому стандартными методами доступа нет, получить можно приведением указателей, но формат приведения может отличаться в разных компиляторах.  

    [vptr|name_|age_|](: Person)  [[vptr|name_|age_|](:Person)|uni_:Student]   (vptr у каждого свой)

∙Вызов виртуального метода — это вызов метода по адресу из таблицы (в коде сохраняется номер метода в таблице).

    p->occupation (); // p->vptr[1](); // т.о. если p указывает на Person, то метод возьмется из его поля vptr

## Таблица виртуальных методов 

    struct  Person {
        virtual ~Person () {}                           // чтобы деструктор также выбирался из табл.вирт.мет. в зависимости от вызывующего объекта
        string  name()  const {return  name_;}          // не виртуальный - не идет в vptr таблицу
        virtual  string  occupation () const = 0;
        ...
    };
    
    struct  Student : Person {
        string   occupation () const {return "student";}// виртуальный - потому что у родителя виртуальный
        virtual  int  group() const {return  group_ ;}
        ...
    };
    
    Person
    0 | ~Person     |0xab22|
    1 | occupation  |0x0000|    // абстрактный метод - адреса нет. На самом деле тут адрес специальной функции обработчика,
                                // которую даже можно перехватить (ибо обращение к 0x0000 всегда -> SEGFAULT).
    
    Student
    0 | ~Student    |0xab46|
    1 | occupation  |0xab68|    // добавлена реализация - адрес есть
    2 | group       |0xab8a|
    

## Построение таблицы виртуальных методов

    struct  Person {
        virtual ~Person () {}               // все деструкторы наследников - тоже вирт.
        virtual  string  occupation () = 0; // абстрактный класс
        ...
    };
    
    struct  Teacher : Person {
        string  occupation ()  {...}
        virtual  string  course ()  {...}
        ...
    };
    
    struct  Professor : Teacher {
        string  occupation ()  {...}
        virtual  string  thesis ()  {...}
        ...
    };
    
    
    Person
    0 | ~Person     |0xab20|
    1 | occupation  |0x0000|
    
    Teacher
    0 | ~Teacher    |0xab48|
    1 | occupation  |0xab60|
    2 | course      |0xab84|

    Professor
    0 | ~Professor  |0xaba8|
    1 | occupation  |0xabb4|
    2 | course      |0xab84|                // course не переопределен в классе, поэтому адрес из родителя (!)
    3 | thesis      |0xabc8|

## Виртуальные методы в конструкторе и деструкторе

    struct  Person {
        virtual  string  name() const {return  name_ ;}
        ...                                                 // вирт.деструктор где-то в этом ...
    };
    
    struct  Teacher : Person {
        Teacher(string  const& nm) : Person(nm)
        { cout  << name (); }                               // вызовем в конструкторе вирт.метод
        ...
    };
    
    struct  Professor : Teacher {
        // string  name() const {return "Prof. " + name_ ;}      // лучше так не писать, т.к. обращение к name_ будет нарушением инкупсуляции
        string  name() const {return "Prof. " + Person::name();} // name_ приватное в ...
        ...
    };
    
    Professor p("Stroustrup"); // "Stroustrup" (из базового класса)

\*) Вызовы Professor() -> Teacher() -> Person(). Выполнение Person() -> Teacher() -> Professor().
Казалось бы, все структуры созданы, таблицы вирт.методов тоже - метод Professor::name() доступен. Но это было бы некорректное поведение, т.к. хоть Professor и создан (вызван), но инициализация его еще не завершена. Поэтому принудительно при вызове в конструкторе/деструкторе виртуальные методы ведут себя как не виртуальные.

Реалзиовано это очень просто. Указатель vptr инициализируеются указателем vptr родителя (указывает на таблицу витр. методов родителя). На свою атуальную таблицу от выставляется только при завершении работы конструктора. С деструктором аналогично - только в обратном порядке, в результате вызываются деструкторы от потомков к родителям.

А т.к. в конструкторе все методы ведут себя как невируальные, то становится понятно, как вызвать чисто виртуальный метод. Нужно в конструкторе базового класса вызвать метод, объявленный чисто виртуальным. Но напрямую сделать это компилятор не даст. Поэтому можно сделать так: в конструкторе базового класса вызвать метод базового класса, который в свою очередь вызывает чисто виртуальный метод.  
А ещё можно подменить обработчик такой ситуации, определив в коде нечто вроде

    extern "C"
    {
        void __cxa_pure_virtual()
        {
            cout << "Are you a programmer?" << endl;
        }
    }


# Объектно-ориентированное программирование

## Ещё раз об ООП

Объектно-ориентированное программирование — концпеция программирования, основанная на понятиях объектов и классов.  
Основные принципы:  

∙инкапсуляция (объединение данные с методами их обработки)  
∙наследование (возможность создавать подклассы, производные классы на основе базовых)  
∙полиморфизм (связанная с наследованием возможность работать с подклассами через ссылку на базовый класс)  
∙абстракция (возможность сокрытия реализации вызывающего класса, чтоб вызывающий код не знал о деталях реализации).  

Подробнее о принципах проектирования ООП-программ можно узнать по ключевым слову „шаблоны проектирования”.

## Как правильно построить иерархию?

Иерархия геометрических фигур: Shape -> (Triangle, Circle, Rectangle).  
Куда добавить класс Square?

1. Квадрат — это прямоугольник, у которого все стороны равны, тогда Shape -> (Triangle, Circle, Rectangle -> (Square, ))  
Т.е. исходим из геометрии объектов, что квадрат это частный случай прямоугольника. Тогда можно представить такой код, который будет работать некорректно:

    void  double_width(Rectangle & r) {     // ожидается, что изменится только ширина
        r.set_width(r.width () * 2);        // но для квадрата - это некорректное поведение
    }

2. Прямоугольник задаётся двумя сторонами, а квадрат — одной, тогда Shape -> (Triangle, Circle, Square -> (Rectangle, ) )  
Т.е. исходим из программирования объектов, квадрату для задания нужна одна сторона. Тогда можно представить такой некорр. код:

    double  area(Square  const& s) {        // если вызвать его для прямоугольник,
        return s.width() * s.width ();      // то получим опять не то, что задумано
    }

3. Правильное решение — не связывать квадрат и прямоугольник наследованием, сделать эти классы независимыми: Shape -> (Triangle, Circle, Square, Rectangle)

Понятно, что пример более чем искусственный, реально все варианты допустимы и определяются конкретной реализацией.


## Агрегирование vs наследование

∙Агрегирование — это включение объекта одного класса в качестве поля в другой.  
∙Наследование устанавливает более сильные связи между классами, нежели агрегирование:  
    
    ∙приведение между объектами,
    ∙доступ к protected членам.

∙Если наследование можно заменить легко на агрегирование, то это нужно сделать.

Примеры некорректного наследования:
    
    ∙Класс Circle унаследовать от класса Point - сложно будет определить функцию расстояния, кроме расстояния между центрами  
    ∙Класс LinearSystem унаследовать от класса Matrix - сложно/невозможно определить операции с матрицами, типа транспонирования, так, чтобы они были корректны для СЛАУ, т.к. они не будут включать правую часть

\*) часто используется правило: "если a имеет b " -  агрегирование, "если a является b" - наследование.

## Принцип подстановки Барбары Лисков 

Liskov Substitution Principle (LSP) Функции, работающие с базовым классом, должны иметь возможность работать с подклассами не зная об этом.  
*) если есть код, работающий с базовым классом, и мы передадим ему унаследованный класс, то работоспособность не должна нарушиться.

Этот принцип является важнейшим критерием при построении иерархий наследования.  
*) т.е. при решении о наследовании, надо подумать, а будет ли корректно (и без костылей) работать уже созданный код с унаследованным классом.

Другие формулировки:  

∙Поведение наследуемых классов не должно противоречить поведению, заданному базовым классом.  
∙Подкласс не должен требовать от вызывающего кода больше, чем базовый класс, и не должен предоставлять вызывающему коду меньше, чем базовый класс.

Пример:  
допустим, у нас есть класс Квадрат, унаследованный от класса Прямоугольник и в классе Прямоугольник определён метод, который увеличивает его ширину в 2 раза. Теперь предположим, что мы хотим переопределить этот метод для класса Квадрат. Мне приходят в голову лишь 3 способа его реализации и все они плохие:  
Способ 1 - увеличить не только ширину квадрата, но и его высоту. В таком случае поведение переопределённого метода начинает противоречить поведению базового - пользователь метода ждёт, что у объекта увеличится размер только ширины, а тут, внезапно, изменится ещё и высота.  
Способ 2 - ничего не делать, вместо этого выкинуть исключение, вернуть код ошибки или что-то наподобие. Тогда получается, что мы предоставляем меньше функций, чем в базовом классе.  
Способ 3 - оставить реализацию из класса Прямоугольник, тем самым нарушив инвариант класса. Самый плохой вариант, делать так не стоит никогда - на выходе пользователь получит объект класса Квадрат, который на самом деле уже не квадрат. Заодно скорее всего сломаются другие методы, определённые в Квадрате - например, метод, вычисляющий площадь. По сути, пользователю придётся всё время помнить, что этот метод можно использовать для прямоугольников, но ни в коем случае не для квадратов.

## Вернемся к иерархии Expression. 

На каждую операцию над объектами иерархии мы могли бы заводить по отдельному виртуальному методу, как мы это сделали с методом evaluate, но такой подход при большом количестве операций приведёт к значительному увеличению размера кода. Вместо этого мы могли бы определить какой-нибудь универсальный интерфейс, который позволял бы реализовывать операции, не раздувая интерфейс классов иерархии Expression. Например это можно сделать используя паттерн посетитель (так же известный как Visitor).

Для этого давайте заведем абстрактный класс Visitor:

    struct Visitor {
        virtual void visitNumber(Number const * number) = 0;
        virtual void visitBinaryOperation(BinaryOperation const * binary) = 0;
        virtual ~Visitor() { }
    };

И добавим в нашу иерархию методы visit:

    struct Expression {
        /* ... */
        virtual void visit(Visitor * visitor) const  = 0;
        /* ... */
    };

    struct Number : Expression {
        /* ... */
        void visit(Visitor * visitor) const { visitor->visitNumber(this); }
        /* ... */
    };


    struct BinaryOperation : Expression {
        /* ... */
        void visit(Visitor * visitor) const { visitor->visitBinaryOperation(this); }
        /* ... */
    };


Для того, чтобы Visitor мог сделать что-либо полезное, мы должны добавить в иерархию Expression методы доступа к свойствам классов (дабы объект класса Visitor мог к ним обратиться). Давайте определим метод для доступа к значению внутри объекта Number, а также методы для доступа к левому и правому операндам объекта BinaryOperation. Это будет выглядеть следующим образом: 
 

    struct Number : Expression {
        /* ... */
        double get_value() const { return value; }
        /* ... */
    };

    struct BinaryOperation : Expression {
        /* ... */
        Expression const * get_left()  const { return left; }
        Expression const * get_right() const { return right; }
        char get_op() const { return op; }
        /* ... */
    };


Теперь для того, чтобы определить какое-либо действие с арифметическим выражением, достаточно просто создать наследника класса Visitor и реализовать в нем методы visitNumber и visitBinaryOperation, которые реализуют требуемые действия. Например, напечатать все бинарные операции, используемые в выражении, можно с помощью следующего наследника класса Visitor:

    struct PrintBinaryOperationsVisitor : Visitor {
        void visitNumber(Number const * number)
        { }

        void visitBinaryOperation(BinaryOperation const * bop)
        {
            bop->get_left()->visit(this);
            std::cout << bop->get_op() << " ";
            bop->get_right()->visit(this);
        }
    };


После чего его можно использовать так:

    Expression const * expr = get_expression();
    PrintBinaryOperationsVisitor visitor;
    expr->visit(&visitor);

Данный код рекурсивно обойдёт дерево, соответствующее арифметическому выражению, и напечатает все бинарные операции в порядке обхода.

# Особенности наследования в C++

## Модификаторы при наследовании

При наследовании можно использовать модификаторы доступа:

    struct A {};
    struct  B1 : public A {};       // любой код, работающий с B1 знает, что он унаследован от A, т.е. может вызывать все методы, 
                                    // унаследованные B1 от A, а также приводить указатели и ссылки на B1 к типу ссылок на A
    struct  B2 : private A {};      // только внутри класса B2 можно обращаться к полям и методам, унаследованным от A, а также 
                                    // приводить указатели и ссылки к типу базового класса А
    struct  B3 : protected A {};    // только внутри B2 и его наследников можно обращаться к полям и методам, унаследованным от A, а также 
                                    // приводить указатели и ссылки к типу базового класса А
    
*) На пальцах если, модификаторы наследования повышают строгость модификаторов доступа до своего уровня, если они ниже, и передают такое поведение по цепочке наследования (все поля/методы становятся не ниже модификатора наследования)

**) модификатор наследования не влияет на то, как дочерний класс получает доступ к членам родительского класса! Это влияет только на то, как другими объектами осуществляется доступ к этим членам через дочерний класс:

Для классов, объявленных как struct, по-умолчанию используется public, для объявленных как class — private (аналогично модификаторм доступа к полям и методам).

Важно: отношение наследования (в терминах ООП) задаётся только public-наследованием.

Использование private- и protected-наследований не вполне соответствует концепции ООП, их целесообразно использовать, если необходимо не только агрегировать другой класс, но и переопределить его виртуальные методы.

## Переопределение private виртуальных методов

    struct  NetworkDevice {
        void  send(void * data , size_t  size) {    // публичный невиртуальный (значит нельзя переопределить в наследниках) метод
            log("start  sending");
            send_impl(data , size);                 // использует приватный виртуальный метод
            log("stop  sending");
        }
        ...
    private:            
        virtual  void  send_impl(void * data , size_t  size) {...}  // нельзя обратиться снаружи и из наследников
    };
    
    struct  Router : NetworkDevice {
    private:
        void  send_impl(void * data , size_t  size) {...}           // тут приватность метода не принципиальна для логики шаблона
    };

*) внешний код через любого наследника используя send() всегда будет делать send() из NetworkDevice(), в котором приватный виаруальный метод будет выбираться по VMT из того класса, который использует внешний код.

## Реализация чистых виртуальных методов 

Чистые виртуальные методы могут иметь определения:

    struct  NetworkDevice {
        virtual  void  send(void * data , size_t  size) = 0;
        ...
    };
    
    void  NetworkDevice::send(void * data , size_t  size) {...} // добавить определение чистого вирт.метода снаружи класса
                                                                // через полное имя, поэтому не попадет в VMT
    
    struct  Router : NetworkDevice {
    private:
        void  send(void * data , size_t  size) {                
            // невиртуальный вызов                              // виртуальный вызов попал бы на обработчик ошибки вызова чист.вирт.мет. в VMT
            NetworkDevice::send(data , size);                   // а вызов через полное имя всегда невиртуальный (хардкодится компилятором)
        }
    };

*) зачем это может понадобиться? Задать поведение по умолчанию, т.е. если в производном классе нет полноценного переопределения виртуального метода send, то можно так поставить более менее рабочую заглушку, типа того. Программа не крашится, а делает чето осмысленное.  
Или, это нужно, когда мы хотим заставить наследников определять этот метод самостоятельно, но при этом если уж совсем не хочется, то есть стандартная реализация

**) внутри вызова NetworkDevice::send() из Router указатель this будет указывать на объект типа Router, а тип указателя изменится на NetworkDevice.

## Интерфейсы

Интерфейс — это абстрактный класс, у которого отсутствуют поля, а все методы являются чистыми виртуальными.  
При этом определн должен быть только деструктор, остальные - только объявлены

    struct  IConvertibleToString {
        virtual ~IConvertibleToString () {}
        virtual  string  toString () const = 0;
    };
    
    struct  IClonable {
        virtual ~IClonable () {}
        virtual  IClonable * clone () const = 0;                // ожидает в потомках метод clone, возвращающий указатель типа базового класса
    };
    
    struct  Person : IClonable {
        Person * clone() { return  new  Person (*this ); }      // например так, тип будет приведен к типу *this автоматический
    };

*) Зачем нужно? Задать единообразный интерфейс же

## Множественное наследование 

В C++ разрешено множественное наследование.

    struct  Person  {};
    struct  Student : Person  {};
    struct  Worker   : Person  {};
    struct  WorkingStudent : Student , Worker  {};
    
*) чревато проблемами, т.к. в WorkingStudent будет храниться две копии унаследованных Student и Worker полей класса Person, которые к тому же не факт, что будут синхронизированы. Кроме того, будет неоднозначность при выборе метода из Person через WorkingStudent

Поэтому стоит избегать наследования реализаций более чем от одногокласса, вместо этого использовать интерфейсы.

    struct  IWorker  {};
    struct  Worker : Person , IWorker  {};          // унаследуем Person + будем поддерживать интерфейс IWorker
    struct  Student : Person  {};
    struct  WorkingStudent : Student , IWorker  {}  // унаследуем Person + будем поддерживать интерфейс IWorker

*) Минус в том, что два раза писать поддержку интерфейса, плюс в том, что не будет двух копий данных, не будет неоднозначности выбора родительских методов.

Множественное наследование — это отдельная большая тема.

## Выбрать верные утверждения:

    /*Путем долгого тыканья:
        ДА Если в базовом классе виртуальная функция определена как public, то в производном классе её можно переопределить как private.
            (как хошь так и определяй, если имеешь доступ)
        ДА Если в базовом классе виртуальная функция определена как private, то в производном классе её можно переопределить как public.
            (как хошь так и определяй, если имеешь доступ)
        ДА Чистый виртуальный метод с определением — это не то же самое, что обычный виртуальный метод.
            (это статический метод по сути, а не динамический)
        нет Производные классы не видят protected-предков своего (непосредственного) базового класса, если они унаследованы от базового класса с модификатором private.
            (производные private наследники имеют эксклюзивный доступ, а protected - только производные имет доступ => они то как раз и имеют, а больше никто)
        нет У интерфейсов нет таблиц виртуальных методов.
            (у всех классов с виртуальными методами есть)
        нет Производные классы не могут переопределять private-виртуальные методы базового класса, если они унаследованы от базового класса с модификатором private.
            (как раз только производные и имеют доступ к базовому если наследовались private)
        ДА Производные классы не видят private-предков своего базового класса. 
            (кто private наследовался, тот только и имеет доступ)
    */